<a href="https://colab.research.google.com/github/toshiikuoo/puclic/blob/master/%E6%A0%AA%E4%BE%A1%E7%9B%B8%E9%96%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#必要なライブラリimport
!pip install lxml html5lib beautifulsoup4

import pandas as pd
from pandas import Series,DataFrame
from pandas_datareader import DataReader

import numpy as np

from datetime import datetime

from scipy.stats.stats import pearsonr
import itertools

In [61]:
#S&P全銘柄のリストを作成
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list=pd.read_html(url)[0].Symbol
sp500_list

0       MMM
1       ABT
2      ABBV
3      ABMD
4       ACN
       ... 
500     XYL
501     YUM
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [0]:
sp500_list_first=sp500_list[0]

In [0]:
#銘柄シンボルのみを取り出したSeriesを作成
sp500_symbol=sp500_list_first["Symbol"]

In [0]:
#listに変換
sp500_symbol_list=sp500_symbol.values.tolist()

In [31]:
#yahooでは銘柄シンボルの"."は"-"に変えないとエラーになるので置換
#"."を含む要素を確認
sp500_symbol_list_in=[i for i in sp500_symbol_list if "." in i]
sp500_symbol_list_in

['BRK.B', 'BF.B']

In [0]:
#"."を"-"に置換
sp500_symbol_list_yahoo=[i.replace(".","-")for i in sp500_symbol_list]

In [34]:
#確認のため"-"の含まれる要素を表示"
sp500_symbol_list_yahoo_in=[i for i in sp500_symbol_list_yahoo if "-" in i]
print(sp500_symbol_list_yahoo_in)

['BRK-B', 'BF-B']


In [39]:
#sp500銘柄の終値を１つのDataFrameに格納
try:
  close_sp500_list=DataReader(sp500_symbol_list_yahoo,'yahoo',datetime(2019, 9, 26, 7, 10, 37, 334315),datetime(2019, 10, 26, 7, 10, 37, 334315),)["Adj Close"]
except KeyError:
  print('No such key')

No such key


In [55]:
#株のシンボルのlistを作成
columns=close_sp500_list.columns.to_list()
len(columns)

55

In [53]:
len(list(itertools.combinations(columns,2)))

1485

In [0]:
#列ごとのペアで相関を算出



#算出した相関を入れるための辞書型作成
correlations={}

#相関を算出
for cola,colb in itertools.combinations(columns,2):
  nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
  correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])

In [41]:
#出力された結果"correlations"はリスト形式なのでDataFrameに変換
result=DataFrame.from_dict(correlations,orient='index')
result.columns=['PCC','p-value']
print(result.sort_values('PCC'))

                 PCC       p-value
TRV__VRTX  -0.885004  4.504950e-08
TRV__WCG   -0.850925  5.209412e-07
VRTX__YUM  -0.849500  5.693920e-07
VRSK__VRTX -0.842442  8.730565e-07
UAA__YUM   -0.827421  2.031590e-06
...              ...           ...
UNP__XYL    0.966105  3.140892e-13
URI__WY     0.969397  1.145911e-13
URI__XYL    0.970509  7.951287e-14
UNP__UTX    0.977705  4.996477e-15
UAA__UA     0.992748  7.047567e-20

[1485 rows x 2 columns]
